In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# StageLoader: store_orders_Stage Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_store_orders` (
         `bg_loadtimestamp`
        ,`bg_sourcesystem`
        ,`total`
        ,`store_name`
        ,`address`
        ,`latitude`
        ,`longitude`
        ,`order_status`
        ,`order_count`
        ,`total_sales`
    )
    SELECT
         {bg_loadtimestamp} AS `bg_loadtimestamp`
        ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`total` AS `total`
        ,`bg_source`.`store_name` AS `store_name`
        ,`bg_source`.`address` AS `address`
        ,`bg_source`.`latitude` AS `latitude`
        ,`bg_source`.`longitude` AS `longitude`
        ,`bg_source`.`order_status` AS `order_status`
        ,`bg_source`.`order_count` AS `order_count`
        ,`bg_source`.`total_sales` AS `total_sales`
    FROM `{marc_databricks_initiative#stage#database_name}`.`{marc_databricks_initiative#stage#schema_name}`.`stg_st_store_orders_source` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['reloadtarget_{marc_databricks_initiative#stage#database_name}_{marc_databricks_initiative#stage#schema_name}_stg_st_store_orders'] = operation_metrics

except Exception as e:
    info('STG_ST_store_orders_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
